# Práctica 4: Visualización de información con plotly

El objetivo de esta práctica es realizar algo parecido a lo que se hizo en la práctica pasada, donde se infirieron comportamientos y tendencias a partir de obtener datos estadísticos descriptivos. En lo que respecta a esta practica es inferir comportamiento o tendencias a partir de visualizar los datos haciendo uso de la librería [**plotly**](https://plot.ly/d3-js-for-python-and-pandas-charts/).

Para utilizar el menor numero de graficas posibles se **agrupan los niveles del factor número de neuronas por cada nivel del factor número de capas**. De esta forma solo se deben generan un numero de graficas igual a la combinación de los niveles de los factores restantes, sin tomar en cuenta el factor partición, porque el objetivo es obtener la combinación de niveles de factores que tengan mejor rendimiento por partición.

In [1]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
from scipy import special
import pandas as pd
import ssl
py.offline.init_notebook_mode(connected=True)

Para ejemplificar el trabajo y evitar ser repetitivo se toman los datos para el método de solución **lbfgs**. Para este caso los factores son:

In [72]:
d = pd.read_csv("lbfgs_crudo.csv")                # carga de datos
for i in d.columns:
    print('factor ',i,' niveles ',d[str(i)].unique())

factor  Unnamed: 0  niveles  [ 1728  1729  1730 ... 29565 29566 29567]
factor  partition  niveles  [10 20 30 40 50 60 70 80 90]
factor  layer_num  niveles  [ 1  4  7 10]
factor  neurona_num  niveles  [ 30  90 150 210]
factor  activation  niveles  ['logistic' 'tanh']
factor  alpha  niveles  [1.e-04 1.e+00 1.e+04]
factor  train_accuracy  niveles  [0.92783333 0.95016667 0.802      ... 0.14612963 0.99957407 0.53668519]
factor  training_time  niveles  [ 23.15649223  21.88026476   6.25390673 ... 608.63541865 667.52977705
  82.80284572]
factor  test_accuracy  niveles  [0.8789 0.8949 0.8065 ... 0.9639 0.1556 0.5359]


Se observa que los factores **número de capas** y **número de neuronas** son los que tienen más niveles, de modo que el numero de neuronas se agruparan por cada nivele del factor numero de capas. Los factores restantes son **alpha** y **función de activación ** cuyos numero de niveles son 3 y 2 respectivamente, por lo tanto, deben generar 6 graficar para cada nivele de partición, y a partir de estas inferir cual combinación de niveles obtiene los mejores rendimientos. para el primer nivel del factor partición se tiene lo siguiente:

In [73]:
figs=dict()
for p in d.activation.unique():
    for q in d.alpha.unique(): 
        neuronas=dict()                                   # el diccionario neuronas tiene como etiquetas los niveles del factor
        for i in d.neurona_num.unique():                  # numero de neuronas y en ellas se guardan los datos cuyo nivel de 
            neuronas[str(i)+' neuronas']=list()           # numero de neuronas es la etiqueta

        x=list()                                          # x es una lista con el valor del eje equis "x", en este caso los
                                                          # niveles del factor numero de capas 
        for i in d.neurona_num.unique():
            for j in d.layer_num.unique():
                for k in range(len(d.test_accuracy[(d.partition==10) & (d.activation==p) & (d.alpha==q) &\
                                         (d.neurona_num==i) & (d.layer_num==j)])):
                    if j==1:
                        x.append(str(j)+' capa')          # se agrega a x el nivel del numero de capas
                    else:
                        x.append(str(j)+' capas')
                for k in d.test_accuracy[(d.partition==10) & (d.activation==p) & (d.alpha==q) &\
                                         (d.neurona_num==i) & (d.layer_num==j)]:
                    neuronas[str(i)+' neuronas'].append(k) # se agrega a la etiqueta ["nivel" de neuronas] el rendimiento en la 
                                                           # fase de prueba que corresponde con el nivel de neuronas y el nivel
                                                           # del numero de capas

        traces=dict()                                      # se ponen en formato caja los datos de rendimiento por cada nivel 
        for i in d.neurona_num.unique():                   # de numero de neuronas en cada nivel de numero de capas
            traces[str(i)]=go.Box(
            y=neuronas[str(i)+' neuronas'],
            x=x,
            name=str(i)+' neuronas')


        datos = list()
        for i in traces:                                   # se juntan todos los formatos caja generados anteriormente
            datos.append(traces[i])

        layout = go.Layout(                                                 # se le da formato a la grafica de salida
            title='particion=10%, activacion='+str(p)+', alpha='+str(q),    # en este caso solo se tiene que especificar
            yaxis=dict(title='presicion de acierto',zeroline=False),        # en el titulo de cada grafica cuales son
            boxmode='group'                                                 # los valores de los demas factores y poder
            )                                                               # diferenciar cada grafica
        figs['10_'+str(p)+'_'+str(q)] = go.Figure(data=datos, layout=layout)

for i in figs:
    py.offline.iplot(figs[i])
# este codigo nos permite graficar cada una de las combinaciones posibles para un nivel de particion
# debido a la incapacidad de graficar mas en mas de tres dimensiones, para que tuviera sentido se agruparon los factores
# con mayor numero de niveles y se hicieron un numero de graficas igual a la combinacion de los factores restantes

Este mismo proceso se repite para los siguientes niveles del factor partición.

## Conclusiones:

* A medida que se aumenta el tamaño de la muestra de entrenamiento aumenta el rendimiento.
* Aumentar el numero de capas tiene un efecto negativo sobre el rendimiento y el nivel que obtiene mejores resultados es con una capa
* Al utilizar la función de activación **logistic** los mejores resultados se obtienen con los valores de **alpha** 0.0001 y 1. Parece no haber diferencia entre usar 150 o 210 neuronas por capa.
* Al utilizar la función de activación **tanh** los mejores resultados se obtienen con los valores de **alpha** 0.0001 y 1. En el caso del numero de neuronas por capa se observa que el nivel con mejores rendimientos es el de 210.

Algunas de las conclusiones de esta práctica confirman las conclusiones de la practica anterior e incluso es posible inferir mas cosas. Esto nos ayudara para establecer las pruebas de hipótesis de la siguiente práctica.